In [1]:
pip install --quiet -U openai ipywidgets langchain langchain-community langchain-core langchain-chroma langchain-openai langchain-huggingface pypdf pysqlite3-binary sentence-transformers 


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Set Up The Model
In this block, we install chromadb and other dependancies.  Chroma requires sqlite3 so that is imported as well.

The LLM that is used is Mistral:Instruct that is hosted by an Ollama container running in OpenShift.

HuggingFace Embeddings are used since they can be run locally and can be configured to take advantage of available GPUs.

In [ ]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

import bs4
import os.path
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.utils.math import cosine_similarity
from langchain_text_splitters import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from typing import List
from IPython.display import display, Markdown

model = ChatOpenAI(model="gemma-2b-it",
                   base_url="https://gemma-2b-it-hosted-llm.apps.sno.brunell.lab/v1",
                   temperature = 0,
                   api_key = "EMPTY",
                   streaming=True  # ! important
                   )

# Gather Data, Chunk it and Store it in the vector store

If the database is not present, then create it by downloading and chunking the files.  If it is present, then just load it.

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs={'device': 'cuda'})
embedding_function = SentenceTransformerEmbeddingFunction()

In [4]:
persist_dir = "db_rhel"

check_file = "False"

path = 'db_rhel/chroma.sqlite3'

check_file = os.path.isfile(path)

if check_file is False:
    urls = [
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/interactively_installing_rhel_from_installation_media/Red_Hat_Enterprise_Linux-9-Interactively_installing_RHEL_from_installation_media-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/automatically_installing_rhel/Red_Hat_Enterprise_Linux-9-Automatically_installing_RHEL-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/upgrading_from_rhel_8_to_rhel_9/Red_Hat_Enterprise_Linux-9-Upgrading_from_RHEL_8_to_RHEL_9-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/configuring_basic_system_settings/Red_Hat_Enterprise_Linux-9-Configuring_basic_system_settings-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/security_hardening/Red_Hat_Enterprise_Linux-9-Security_hardening-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/composing_a_customized_rhel_system_image/Red_Hat_Enterprise_Linux-9-Composing_a_customized_RHEL_system_image-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/configuring_and_managing_networking/Red_Hat_Enterprise_Linux-9-Configuring_and_managing_networking-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/configuring_and_managing_cloud-init_for_rhel_9/Red_Hat_Enterprise_Linux-9-Configuring_and_managing_cloud-init_for_RHEL_9-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/using_image_mode_for_rhel_to_build_deploy_and_manage_operating_systems/Red_Hat_Enterprise_Linux-9-Using_image_mode_for_RHEL_to_build_deploy_and_manage_operating_systems-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/configuring_and_managing_virtualization/Red_Hat_Enterprise_Linux-9-Configuring_and_managing_virtualization-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/building_running_and_managing_containers/Red_Hat_Enterprise_Linux-9-Building_running_and_managing_containers-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_enterprise_linux/9/pdf/9.4_release_notes/Red_Hat_Enterprise_Linux-9-9.4_Release_Notes-en-US.pdf'
    ]
    
    pages = []
    
    for file in urls:
        loader = PyPDFLoader(file, extract_images=False)
        pages = pages + loader.load()
        
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=1275, chunk_overlap=0)
    
    splits = text_splitter.split_documents(pages)
    
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_dir)

In [5]:
persist_dir = "db_ansible"

check_file = "False"

path = 'db_ansible/chroma.sqlite3'

check_file = os.path.isfile(path)

if check_file is False:
    urls = [
        'https://docs.redhat.com/en-us/documentation/red_hat_ansible_automation_platform/2.4/pdf/getting_started_with_automation_controller/Red_Hat_Ansible_Automation_Platform-2.4-Getting_started_with_automation_controller-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_ansible_automation_platform/2.4/pdf/getting_started_with_automation_hub/Red_Hat_Ansible_Automation_Platform-2.4-Getting_started_with_automation_hub-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_ansible_automation_platform/2.4/pdf/getting_started_with_ansible_playbooks/Red_Hat_Ansible_Automation_Platform-2.4-Getting_started_with_Ansible_Playbooks-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_ansible_automation_platform/2.4/pdf/containerized_ansible_automation_platform_installation_guide/Red_Hat_Ansible_Automation_Platform-2.4-Containerized_Ansible_Automation_Platform_installation_guide-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_ansible_automation_platform/2.4/pdf/deploying_the_red_hat_ansible_automation_platform_operator_on_openshift_container_platform/Red_Hat_Ansible_Automation_Platform-2.4-Deploying_the_Red_Hat_Ansible_Automation_Platform_operator_on_OpenShift_Container_Platform-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_ansible_automation_platform/2.4/pdf/red_hat_ansible_automation_platform_operations_guide/Red_Hat_Ansible_Automation_Platform-2.4-Red_Hat_Ansible_Automation_Platform_operations_guide-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/red_hat_ansible_automation_platform/2.4/pdf/red_hat_ansible_automation_platform_hardening_guide/Red_Hat_Ansible_Automation_Platform-2.4-Red_Hat_Ansible_Automation_Platform_hardening_guide-en-US.pdf'
    ]

    pages = []
    
    for file in urls:
        loader = PyPDFLoader(file, extract_images=False)
        pages = pages + loader.load()
        
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=1700, chunk_overlap=0)
    
    splits = text_splitter.split_documents(pages)
    
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_dir)

In [6]:
persist_dir = "db_openshift"

check_file = "False"

path = 'db_openshift/chroma.sqlite3'

check_file = os.path.isfile(path)

if check_file is False:
    urls = [
        'https://docs.redhat.com/en-us/documentation/openshift_container_platform/4.16/pdf/installing/OpenShift_Container_Platform-4.16-Installing-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/assisted_installer_for_openshift_container_platform/2024/pdf/installing_openshift_container_platform_with_the_assisted_installer/Assisted_Installer_for_OpenShift_Container_Platform-2024-Installing_OpenShift_Container_Platform_with_the_Assisted_Installer-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/openshift_container_platform/4.16/pdf/authentication_and_authorization/OpenShift_Container_Platform-4.16-Authentication_and_authorization-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/openshift_container_platform/4.16/pdf/storage/OpenShift_Container_Platform-4.16-Storage-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/openshift_container_platform/4.16/pdf/scalability_and_performance/OpenShift_Container_Platform-4.16-Scalability_and_performance-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/openshift_container_platform/4.16/pdf/cli_tools/OpenShift_Container_Platform-4.16-CLI_tools-en-US.pdf',
        'https://docs.redhat.com/en-us/documentation/openshift_container_platform/4.16/pdf/virtualization/OpenShift_Container_Platform-4.16-Virtualization-en-US.pdf'
    ]

    pages = []
    
    for file in urls:
        loader = PyPDFLoader(file, extract_images=False)
        pages = pages + loader.load()
        
    text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=675, chunk_overlap=0)
    
    splits = text_splitter.split_documents(pages)
    
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_dir)

# Set up the prompts

In [7]:
# Set up the prompts

rhel_template = """You are an expert in Red Hat Enterprise Linux (RHEL) who retrieves information from documents.\n
When you don't know the answer say that you don't know.\n
Always include a list of "SOURCES" part in your answer with a link to the source document.\n

QUESTION: {question}
=========
{source_docs}
=========
ANSWER: """

ansible_template = """You are an expert in Ansible Automation Platform (AAP) who retieves information from documents.\n
When you don't know the answer say that you don't know.\n
Always include a list of "SOURCES" part in your answer with a link to the source document.\n

QUESTION: {question}
=========
{source_docs}
=========
ANSWER: """

openshift_template = """You are an expert in OpenShift Container Platform (OCP), and it's CLI tool oc.\n
You retrieve information from documents.\n
When you don't know the answer say that you don't know.\n
Always include a list of "SOURCES" part in your answer with a link to the source document.\n

QUESTION: {question}
=========
{source_docs}
=========
ANSWER: """

other_template = """You are an expert in general knowledge.\n
Do not answer the question.\n
Ask how the questions is related a Red Hat product.\n
"""

prompt_templates = [rhel_template, ansible_template, openshift_template, other_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# these lists are used to 
rhel_topics = """Red Hat Enterprise Linux, RHEL"""
ansible_topics = """Ansible Automation Platform, AAP"""
openshift_topics = """OpenShift Container Platform, OCP, oc, vrtctl"""
other_topics = """everything else, satellites, food, fictional characters"""

topic_templates = [rhel_topics, ansible_topics, openshift_topics, other_topics]
topic_embeddings = embeddings.embed_documents(topic_templates)

# Set up some defauls
rag_prompt = ChatPromptTemplate.from_template(ansible_template)

# Define the RAG Chains as functions.

In [8]:
# the prompt_router function detemines which product the question is most likely about
# and returns the name of the product

def prompt_router(input):
    query_embedding = embeddings.embed_query(input["question"])
    
    similarity = cosine_similarity([query_embedding], topic_embeddings)[0]
    # enable this line if you want to observe similarity scoring
    # print("===== Similarity:   (RHEL       Ansible    OpenShift   Other)\n          Scores: ",similarity,"\n")
    most_similar = prompt_templates[similarity.argmax()]
    
    if most_similar == rhel_template:
        return "rhel"
    elif most_similar == ansible_template:
        return  "ansible"
    elif most_similar == openshift_template:
        return "openshift"
    elif most_similar == other_template:
        return "other"

# Format the source docs forthe LLM
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(f"Content: {doc.page_content}\nSource: {doc.metadata['source']}" for doc in docs)

# Chains

# Process the source_docs to generate the answer
def rag_chain_fn(question):
    rag_from_docs_chain = (
        RunnablePassthrough.assign(
            source_docs=(lambda x: format_docs(x["source_docs"]))
        )
        | rag_prompt
        | model
        | StrOutputParser()
    )
    
    # Retrieve source docs and invoke the last chain.
    rag_chain = (RunnableParallel(
        {
            "source_docs": lambda x: retriever,
            "question": RunnablePassthrough()
        }
    ).assign(answer=rag_from_docs_chain))
    
    # Stream the output
    #curr_key = None
    #for chunk in rag_chain.stream(question):
    #    for key in chunk:
    #        if key == "answer":
    #            print(chunk[key], end="", flush=True)
    
    # Uncomment this line (and remove the streaming section) to return to normal output.
    return (rag_chain.invoke(question))


# The router_chain calls the prompt-router function to determine the topic of the question.
def router_chain_fn(question):
    
    router_chain = (
        {"question": RunnablePassthrough()}
        | RunnableLambda(prompt_router)
    )
    
    return (router_chain.invoke(question))

# Main Loop
Here we get the question, set up some variables and route to the correct function to obtain results.

In [9]:
# Main loop - get the question, figures out the topic, routes to the right db and 
# then works to generate the answer.

question = "Tell me about the security features in RHEL."

product = router_chain_fn(question)

db_name = "db_" + product

template = eval(product + "_template")

rag_prompt = ChatPromptTemplate.from_template(template)
vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)
retriever = vectorstore.as_retriever()

results = rag_chain_fn(question)

# Uncomment the below lines to revert to normal output.
answer = results["answer"]
    
display(Markdown(answer))

PermissionDeniedError: RBAC: access denied